## 2.4 Tensor Flow Project (IMAGE): MLP with MNIST - Autoencoder

__Network Strucuter:__ 1 Hidden Layer, Input -> Hidden(encoding), Hidden -> Output (decoding)

In [10]:
%matplotlib inline

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [11]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('..\\ClassSampleData\\', validation_size=0)

Extracting ..\ClassSampleData\train-images-idx3-ubyte.gz
Extracting ..\ClassSampleData\train-labels-idx1-ubyte.gz
Extracting ..\ClassSampleData\t10k-images-idx3-ubyte.gz
Extracting ..\ClassSampleData\t10k-labels-idx1-ubyte.gz


In [12]:
# Network Structure
# Size of the encoding layer (the hidden layer)
encoding_dim = 32

image_size = mnist.train.images.shape[1]

inputs_ = tf.placeholder(tf.float32, (None, image_size), name='inputs')
targets_ = tf.placeholder(tf.float32, (None, image_size), name='targets')


encoded = tf.layers.dense(inputs_, encoding_dim, activation=tf.nn.relu)
logits = tf.layers.dense(encoded, image_size, activation=None)

decoded = tf.nn.sigmoid(logits, name='output')

loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=targets_, logits=logits)
cost = tf.reduce_mean(loss)
opt = tf.train.AdamOptimizer(0.001).minimize(cost)

In [13]:
sess = tf.Session()
epochs = 20
batch_size = 200

sess.run(tf.global_variables_initializer())
for e in range(epochs):
    for ii in range(mnist.train.num_examples//batch_size):
        batch = mnist.train.next_batch(batch_size)
        feed = {inputs_: batch[0], targets_: batch[0]}
        batch_cost, _ = sess.run([cost, opt], feed_dict=feed)

    print("Epoch: {}/{}...".format(e+1, epochs), "Training loss: {:.4f}".format(batch_cost))

Epoch: 1/20... Training loss: 0.1829
Epoch: 2/20... Training loss: 0.1437
Epoch: 3/20... Training loss: 0.1247
Epoch: 4/20... Training loss: 0.1132
Epoch: 5/20... Training loss: 0.1076
Epoch: 6/20... Training loss: 0.1016
Epoch: 7/20... Training loss: 0.0994
Epoch: 8/20... Training loss: 0.0940
Epoch: 9/20... Training loss: 0.0951
Epoch: 10/20... Training loss: 0.0911
Epoch: 11/20... Training loss: 0.0958
Epoch: 12/20... Training loss: 0.0942
Epoch: 13/20... Training loss: 0.0910
Epoch: 14/20... Training loss: 0.0970
Epoch: 15/20... Training loss: 0.0940
Epoch: 16/20... Training loss: 0.0917
Epoch: 17/20... Training loss: 0.0916
Epoch: 18/20... Training loss: 0.0934
Epoch: 19/20... Training loss: 0.0919
Epoch: 20/20... Training loss: 0.0921


## 2.5 Tensor Flow Project (IMAGE): CNN with MNIST - Autoencoder

### Network Structure:  
__1. Input: (None, 28, 28, 1)__  
__2. Encoding/Convolution/Compression:__   
    2.1. Conv2d, stride=1, padding=same, coding nodes=n, (None, 28, 28, n)  
    2.2. Maxpooling, stride=2, padding=same, (None, 14, 14, n)  
    2.3. Conv2d, stride=1, padding=same, coding nodes=n/2, (None, 14, 14, n/2)  
    2.4. Maxpooling, stride=2, padding=same, (None, 7, 7, n/2)  
    2.5. Conv2d, stride=1, padding=same, coding nodes=n/2, (None, 7, 7, n/2)  
    2.6. Maxpooling, stride=2, padding=same, (None, 4, 4, n/2)  
__3. Decoding/Upsampling/Uncompression:__   
    3.1. Resize, (None, 7, 7, n/2)  
    3.2. Conv2d, stride=1, padding=same, coding nodes=n/2, (None, 7, 7, n/2)  
    3.3. Resize, (None, 14, 14, n/2)  
    3.4. Conv2d, stride=1, padding=same, coding nodes=n/2, (None, 14, 14, n/2)  
    3.5. Resize, (None, 28, 28, n/2)  
    3.6. Conv2d, stride=1, padding=same, coding nodes=n, (None, 28, 28, n)  
__4. Output: (None, 28, 28, 1)__  

__Notes:__ Same data as 2.4  
__Notes2:__ A good use case will be denoise images with deeper layers

In [14]:
inputs_ = tf.placeholder(tf.float32, (None, 28, 28, 1), name='inputs')
targets_ = tf.placeholder(tf.float32, (None, 28, 28, 1), name='targets')

### Encoder
conv1 = tf.layers.conv2d(inputs_, 16, (3,3), padding='same', activation=tf.nn.relu)
# Now 28x28x16
maxpool1 = tf.layers.max_pooling2d(conv1, (2,2), (2,2), padding='same')
# Now 14x14x16
conv2 = tf.layers.conv2d(maxpool1, 8, (3,3), padding='same', activation=tf.nn.relu)
# Now 14x14x8
maxpool2 = tf.layers.max_pooling2d(conv2, (2,2), (2,2), padding='same')
# Now 7x7x8
conv3 = tf.layers.conv2d(maxpool2, 8, (3,3), padding='same', activation=tf.nn.relu)
# Now 7x7x8
encoded = tf.layers.max_pooling2d(conv3, (2,2), (2,2), padding='same')
# Now 4x4x8

### Decoder
upsample1 = tf.image.resize_nearest_neighbor(encoded, (7,7))
# Now 7x7x8
conv4 = tf.layers.conv2d(upsample1, 8, (3,3), padding='same', activation=tf.nn.relu)
# Now 7x7x8
upsample2 = tf.image.resize_nearest_neighbor(conv4, (14,14))
# Now 14x14x8
conv5 = tf.layers.conv2d(upsample2, 8, (3,3), padding='same', activation=tf.nn.relu)
# Now 14x14x8
upsample3 = tf.image.resize_nearest_neighbor(conv5, (28,28))
# Now 28x28x8
conv6 = tf.layers.conv2d(upsample3, 16, (3,3), padding='same', activation=tf.nn.relu)
# Now 28x28x16

logits = tf.layers.conv2d(conv6, 1, (3,3), padding='same', activation=None)
#Now 28x28x1

decoded = tf.nn.sigmoid(logits, name='decoded')

loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=targets_, logits=logits)
cost = tf.reduce_mean(loss)
opt = tf.train.AdamOptimizer(0.001).minimize(cost)

In [15]:
sess = tf.Session()

epochs = 20
batch_size = 200
sess.run(tf.global_variables_initializer())
for e in range(epochs):
    for ii in range(mnist.train.num_examples//batch_size):
        batch = mnist.train.next_batch(batch_size)
        imgs = batch[0].reshape((-1, 28, 28, 1))
        batch_cost, _ = sess.run([cost, opt], feed_dict={inputs_: imgs,
                                                         targets_: imgs})

    print("Epoch: {}/{}...".format(e+1, epochs),
          "Training loss: {:.4f}".format(batch_cost))

Epoch: 1/20... Training loss: 0.1610
Epoch: 2/20... Training loss: 0.1388
Epoch: 3/20... Training loss: 0.1302
Epoch: 4/20... Training loss: 0.1214
Epoch: 5/20... Training loss: 0.1155
Epoch: 6/20... Training loss: 0.1115
Epoch: 7/20... Training loss: 0.1095
Epoch: 8/20... Training loss: 0.1081
Epoch: 9/20... Training loss: 0.1069
Epoch: 10/20... Training loss: 0.1053
Epoch: 11/20... Training loss: 0.1069
Epoch: 12/20... Training loss: 0.1007
Epoch: 13/20... Training loss: 0.0994
Epoch: 14/20... Training loss: 0.1022
Epoch: 15/20... Training loss: 0.0975
Epoch: 16/20... Training loss: 0.1017
Epoch: 17/20... Training loss: 0.0999
Epoch: 18/20... Training loss: 0.1004
Epoch: 19/20... Training loss: 0.1002
Epoch: 20/20... Training loss: 0.0982
